In [12]:
# this file is for model training and testing
# use tf-idf to convert text.
# use Random Forest, SVM and Logistic Regression to train the model and test.

# import
import re
import pandas as pd 
import numpy as np 
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from scipy.sparse import coo_matrix
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier


In [4]:
df = pd.read_csv('DataCleanB_Eng.csv')  # English Data
# df = pd.read_csv('DataCleanB_Arabic.csv')  # Arabi Data


,Number,Topic,Label,Text
0,628949369883000832,@microsoft,negative,dear @Microsoft the newOoffice for Mac is grea...
1,628976607420645377,@microsoft,negative,@Microsoft how about you make a system that do...
2,629023169169518592,@microsoft,negative,I may be ignorant on this issue but... should ...
3,629179223232479232,@microsoft,negative,"Thanks to @microsoft, I just may be switching ..."
4,629226490152914944,@microsoft,positive,"Microsoft, I may not prefer your gaming branch..."
...,...,...,...,...
4304,639166904813223937,metlife,positive,Heading up to MetLife tomorrow for the Jets game
4305,639295526995890177,metlife,positive,Going to MetLife tomorrow but not to see the b...
4306,639804828739346432,metlife,positive,It's the first Football Friday of the year. Th...
4307,639855845958885376,metlife,positive,@Racalto_SK ok good to know. Punting at MetLif...


In [40]:
def tf_idf(contents):
  # word-> matrix: a[i][j] is word frequency of j under text i
  vectorizer = CountVectorizer()

  # Count tf-idf weights for each word
  transformer = TfidfTransformer()

  #fit_transform: tf-idf 
  tfidf = transformer.fit_transform(vectorizer.fit_transform(contents)) # fit_transform: convert text into word-frequency matrix

  # Get all phrases in bag-of-words model 
  word = vectorizer.get_feature_names()
  # print("Length of words:", len(word))
  X = coo_matrix(tfidf, dtype=np.float32).toarray() 


  return X

In [41]:
def KNN_classifier(X_train, X_test, x_val, y_val, y_train, y_test):
  # KNN
  LR = KNeighborsClassifier(n_neighbors=4)
  LR.fit(X_train, y_train)
  val_acc = LR.score(x_val, y_val)
  test_val = LR.score(X_test, y_test)
  train_acc = LR.score(X_train, y_train)
  # print('Validation Accuracy:{}'.format(val_acc))
  # print('Test Accuracy:{}'.format(test_val))
  # print('Train Accuracy:{}'.format(train_acc))
  pre = LR.predict(X_test)
  # print(classification_report(y_test, pre))
  return LR, val_acc, test_val, train_acc

In [31]:
def train_model(df):

  labels = df["Label"].to_list()
  contents = df["Text"].to_list()

  # td-idf
  tf_matrix = tf_idf(contents)

  # split data
  X_train, X_test, y_train, y_test = train_test_split(tf_matrix, labels, test_size=0.4, random_state=1)
  x_val = X_test[10:]
  y_val = y_test[10:]


  # logistic regression
  LR, val_acc, test_val, train_acc = KNN_classifier(X_train, X_test, x_val, y_val, y_train, y_test)

  return val_acc, test_val, train_acc 

In [44]:
topicss = list(set(df["Topic"].to_list()))
print(len(topicss))

arg_val_acc = 0
arg_test_acc = 0
arg_train_acc = 0
for t in topicss:
  df_topic = df[df["Topic"]==t]
  # print(df_topic)
  val_acc, test_val, train_acc = train_model(df_topic)
  arg_val_acc += val_acc
  arg_test_acc += test_val
  arg_train_acc += train_acc
arg_val_acc = arg_val_acc/60
arg_test_acc = arg_test_acc/60
arg_train_acc = arg_train_acc/60
print("arg_val_acc: "+ str(arg_val_acc))
print("arg_test_acc: "+ str(arg_test_acc))
print("arg_train_acc: "+ str(arg_train_acc))

arg_val_acc: 0.8223697493501413
arg_test_acc: 0.834276337263977
arg_train_acc: 0.9088856969542568


In [ ]:
# these are for data_exploration
# please ignore

print("Number of Topics:")
print(len(df["Topic"].value_counts().index))
print()
print("Number of Data for each Topics:")
topics = df["Topic"].value_counts()
# print(topics)
# print(type(topics))
topic_lst = topics[0]
print(topic_lst)

Number of Topics:
60

Number of Data for each Topics:
94
